# **Đề bài**
Viết một chương trình để tính toán điểm thi cho nhiều lớp với sĩ số hàng nghìn học sinh. Mục đích của chương trình giúp giảm thời gian chấm điểm. Áp dụng các functions (hàm) khác nhau trong Python để viết chương trình với các tác vụ sau:
* Mở các tập tin văn bản bên ngoài được yêu cầu với exception-handling
* Quét từng dòng của câu trả lời bài thi để tìm dữ liệu hợp lệ và cung cấp báo cáo tương ứng
* Chấm điểm từng bài thi dựa trên tiêu chí đánh giá (rubric) được cung cấp và báo cáo
* Tạo tập tin kết quả được đặt tên thích hợp

In [2]:
import re
import numpy as np

## *Task 1*
1.1. Tạo một chương trình Python mới có tên “lastname_firstname_grade_the_exams.py.”

1.2. Viết hàm cho phép người dùng nhập tên của một tệp và truy cập đọc.

1.3. Nếu tệp tồn tại, bạn có thể in ra một thông báo xác nhận. Nếu tệp không tồn tại, hiển thị không thể tìm thấy tệp và yêu cầu nhập lại.

1.4. Sử dụng try/except để thực hiện việc này (đừng chỉ sử dụng một loạt câu lệnh “if”):

In [3]:
def read_file():
    while True:
        try:
            global file_name, file
            file_name = input('Enter a class file to grade (i.e. class1 for class1.txt):')
            file_path = './Data Files/Input/' + file_name + '.txt'
            with open(file_path, 'r') as file:
                print('Successfully opened', file_name + '.txt')
                break
        except FileNotFoundError:
            print('File cannot be found.')

## *Task 2*
Tiếp theo, bạn sẽ cần phân tích dữ liệu có trong tệp bạn vừa mở để đảm bảo rằng nó ở đúng định dạng. Mỗi tệp dữ liệu chứa một loạt câu trả lời của học sinh ở định dạng sau:
```
N12345678,B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D
```
hoặc
```
N12345678,B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D,A,B,C,D,E
```
Nhiệm vụ của bạn cho phần này của chương trình là thực hiện như sau:

2.1. Báo cáo tổng số dòng dữ liệu được lưu trữ trong tệp.

2.2. Báo cáo tổng số dòng dữ liệu không hợp lệ trong tệp.

*Một dòng hợp lệ chứa danh sách 26 giá trị được phân tách bằng dấu phẩy
N# cho một học sinh là mục đầu tiên trên dòng. Nó phải chứa ký tự “N” theo sau là 8 ký tự số.*

2.3. Nếu một dòng dữ liệu không hợp lệ, bạn nên báo cáo cho người dùng bằng cách in ra một thông báo lỗi.

**Gợi ý:** Sử dụng phương pháp split để tách dữ liệu ra khỏi tệp.

In [4]:
def report_validity(file):
    print('**** ANALYZING ****')
    list_of_lines = file.read().splitlines() # Get the data of file and split it into a list of line. 
    global n, valid_data
    n = len(list_of_lines)  # The number of students in class.
    valid_data = []        # List of valid lines that can be processed.
    for line in list_of_lines:
        student = line.split(',')
        try:
            # Check if data includes ID and 25 answers.
            if len(student) != 26:  
                raise ValueError('Invalid line of data: does not contain exactly 26 values\n')
            
            # Check if ID starts with 'N' and followed by 8 digits.
            student_ID = student[0]
            if bool(re.match(r'^N\d{8}$', student_ID)) == False:    
                raise NameError('Invalid line of data: N# is invalid\n')
            
        except ValueError as VE:
            print(VE, student)
        except NameError as NE:
            print(NE, student)
        else:
            valid_data.append(student) # Add a student data into list to be graded.
            
    if len(valid_data) == n:
        print('No errors found!')
    print('**** REPORT ****')
    print(f'Total valid lines of data: {len(valid_data)}')
    print(f'Total invalid lines of data: {n - len(valid_data)}')

## *Task 3*
Tiếp theo, bạn sẽ viết chương trình để chấm điểm các bài thi cho một phần nhất định. Kỳ thi gồm 25 câu hỏi, trắc nghiệm. Đây là một chuỗi đại diện cho các câu trả lời:
```
answer_key = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
```
Chương trình sẽ sử dụng những câu trả lời này để tính điểm cho mỗi dòng dữ liệu hợp lệ. Điểm có thể được tính như sau:

* +4 điểm cho mỗi câu trả lời đúng
* 0 điểm cho mỗi câu trả lời bị bỏ qua
* -1 điểm cho mỗi câu trả lời sai
Chúng ta cũng cần thống kê các yêu cầu sau:
* Đếm số lượng học sinh đạt điểm cao (>80).

* Điểm trung bình.

* Điểm cao nhất.

* Điểm thấp nhất.

* Miền giá trị của điểm (cao nhất trừ thấp nhất).

* Giá trị trung vị (Sắp xếp các điểm theo thứ tự tăng dần. Nếu # học sinh là số lẻ, bạn có thể lấy giá trị nằm ở giữa của tất cả các điểm (tức là [0, 50, 100] — trung vị là 50). Nếu # học sinh là chẵn bạn có thể tính giá trị trung vị bằng cách lấy giá trị trung bình của hai giá trị giữa (tức là [0, 50, 60, 100] — giá trị trung vị là 55)).

* Trả về các câu hỏi bị học sinh bỏ qua nhiều nhất theo thứ tự: số thứ tự câu hỏi - số lượng học sinh bỏ qua -  tỉ lệ bị bỏ qua (nếu có cùng số lượng cho nhiều câu hỏi bị bỏ thì phải liệt kê ra đầy đủ).

* Trả về các câu hỏi bị học sinh sai qua nhiều nhất theo thứ tự: số thứ tự câu hỏi - số lượng học sinh trả lời sai - tỉ lệ bị sai (nếu có cùng số lượng cho nhiều câu hỏi bị sai thì phải liệt kê ra đầy đủ).

*Lưu ý: các giá trị số thực làm tròn 3 chữ số thập phân*

In [5]:
def grade():
    global skipped, incorrectly_answered, results
    skipped, incorrectly_answered = [0]*25, [0]*25 # To count the number of students that skipping or answering incorrectly each question.
    results = [[],[]] # Contains list of IDs and list of scores.
    answer_keys = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D".split(',')
    for student in valid_data:
        answers = student[1:] # Student's answers begin from index 1.
        id = student[0]
        score = 0
        for i in range(25):
            # No answer -> +0.
            if answers[i] == '':
                skipped[i] += 1
            # Correct answer -> +4.
            elif answers[i] == answer_keys[i]:
                score += 4
            # Incorrect answer -> -1.
            else:
                score -= 1
                incorrectly_answered[i] += 1
        results[0].append(id)
        results[1].append(score)

In [6]:
def report_results():
    score = np.array(results[1])
    print(f'Total student of high scores: {np.count_nonzero(score > 80)}')
    print(f'Mean (average) score: {round(np.mean(score), 3)}')
    print(f'Highest score: {np.max(score)}')
    print(f'Lowest score: {np.min(score)}')
    print(f'Range of scores: {np.max(score) - np.min(score)}')
    print(f'Median score: {round(np.median(score), 3)}')

    n_1 = max(skipped)
    n_2 = max(incorrectly_answered)

    print('Questions that most people skip:')
    for i in range(25):
        if skipped[i] == n_1:
            print(f'{i + 1} - {n_1} - {round(n_1/n, 3)}')
            
    print('Questions that most people answer incorrectly:')
    for i in range(25):
        if incorrectly_answered[i] == n_2:
            print(f'{i + 1} - {n_2} - {round(n_2/n, 3)}')

## *Task 4:*
Cuối cùng, yêu cầu chương trình là tạo một tệp “kết quả” chứa các kết quả chi tiết cho từng học sinh trong lớp. Mỗi dòng của tệp này phải chứa số ID của học sinh, dấu phẩy và sau đó là điểm của họ. Bạn nên đặt tên tệp này dựa trên tên tệp gốc được cung cấp — ví dụ: nếu người dùng chọn “class1.txt”, bạn nên lưu trữ kết quả trong tệp có tên “class1_grades.txt”.

In [1]:
def write_file():
    content = ''
    for id, score in zip(results[0], results[1]):
        content += (id + ',' + str(score))
        if id != results[0][-1]:
            content += '\n'

    output_path = './Data Files/Output/' + file_name + '_grade.txt'
    with open(output_path, 'w') as fw:
        fw.write(content)

# **Phần chạy chương trình:**

In [11]:
def main():
    read_file()
    with open(file.name, 'r') as f:
        report_validity(f)
        grade()
        report_results()
        write_file()

In [18]:
main()

Successfully opened class8.txt
**** ANALYZING ****
No errors found!
**** REPORT ****
Total valid lines of data: 49
Total invalid lines of data: 0
Total student of high scores: 17
Mean (average) score: 76.673
Highest score: 92
Lowest score: 59
Range of scores: 33
Median score: 77.0
Questions that most people skip:
4 - 9 - 0.184
Questions that most people answer incorrectly:
4 - 9 - 0.184
15 - 9 - 0.184
